<br>
# **Sequence To Sequence Model**
Simple LSTM

<br>
## **1 Introduction**

<img src="https://camo.githubusercontent.com/242210d7d0151cae91107ee63bff364a860db5dd/687474703a2f2f6936342e74696e797069632e636f6d2f333031333674652e706e67">

시퀀스 2 시퀀스 모델의 Tag
1. **S** : **디코딩 입력의 시작**을 나타내는 심볼
1. **E** : **디코딩 출력을 끝**을 나타내는 심볼
1. **P** : 현재 배치 데이터의 time step 크기보다 작은 경우 **빈 시퀀스를 채우는 심볼**

예) 현재 배치 데이터의 최대 크기가 4 인 경우
1.   word -> ['w', 'o', 'r', 'd']
1.   to   -> ['t', 'o', 'P', 'P']

In [10]:
import tensorflow as tf
import numpy as np


# 영어를 한글로 번역하기 위한 학습 데이터
seq_data = [['word', '단어'], ['wood', '나무'],
            ['game', '놀이'], ['girl', '소녀'],
            ['kiss', '키스'], ['love', '사랑']]

char_arr = [c       for c    in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic  = {n: i    for i, n in enumerate(char_arr)}
dic_len  = len(num_dic)
num_dic

{'S': 0,
 'E': 1,
 'P': 2,
 'a': 3,
 'b': 4,
 'c': 5,
 'd': 6,
 'e': 7,
 'f': 8,
 'g': 9,
 'h': 10,
 'i': 11,
 'j': 12,
 'k': 13,
 'l': 14,
 'm': 15,
 'n': 16,
 'o': 17,
 'p': 18,
 'q': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'w': 25,
 'x': 26,
 'y': 27,
 'z': 28,
 '단': 29,
 '어': 30,
 '나': 31,
 '무': 32,
 '놀': 33,
 '이': 34,
 '소': 35,
 '녀': 36,
 '키': 37,
 '스': 38,
 '사': 39,
 '랑': 40}

<br>
## **2 Modeling**
Encoder & Decoder

In [2]:
# Encoder, Decoder 정의
def make_batch(seq_data):
    input_batch, output_batch, target_batch = [], [], []
    for seq in seq_data:
        input_data  = [num_dic[n] for n in seq[0]]         # Encoder Cell
        output_data = [num_dic[n] for n in ('S' + seq[1])] # Decoder Cell ('S'심볼은 시작값)
        target      = [num_dic[n] for n in (seq[1] + 'E')] # Decoder Cell 학습위한 비교셀 ('E'심볼은 종료값)
        input_batch.append(np.eye(dic_len)[input_data])
        output_batch.append(np.eye(dic_len)[output_data])
        target_batch.append(target)
    return input_batch, output_batch, target_batch

In [3]:
# Parametor
learning_rate = 0.01
n_hidden      = 128
total_epoch   = 100
n_class = n_input = dic_len # 입력과 출력이 one-hot 인코딩으로 크기도 같다

# tf.placeholder(tf.float32, [batch size, time steps, input size])
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets   = tf.placeholder(tf.int64,   [None, None])

In [4]:
# 신경망 모델 : Seq2Seq 모델은 인코더와 디코더 형식이 같다.
# 인코더 셀을 구성
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    outputs, enc_states  = tf.nn.dynamic_rnn(enc_cell, enc_input, 
                                             dtype = tf.float32)
# 디코더 셀을 구성
# initial_state = enc_states : 인코더 셀의 최종값을 디코더 셀의 초기 상태값에 입력
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    outputs, dec_states  = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                             initial_state = enc_states,
                                             dtype         = tf.float32)

In [5]:
model = tf.layers.dense(outputs, n_class, activation=None)
cost  = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = model, 
                labels = targets))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

<br>
## **3 Model Training**
신경망 모델 학습

In [6]:
# Traing Model
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  targets: target_batch})
    if epoch % 15 == 0 :
        print('Epoch: {:3} cost: {:.6f}'.format(epoch + 1, loss))

Epoch:   1 cost: 3.763897
Epoch:  16 cost: 0.019405
Epoch:  31 cost: 0.003713
Epoch:  46 cost: 0.002375
Epoch:  61 cost: 0.000587
Epoch:  76 cost: 0.000586
Epoch:  91 cost: 0.000378


<br>
## **4 model Prediction**
번역 테스트

In [ ]:
# 모르는 단어는 ['word', 'PPPP']로 출력한다
def translate(word):
    seq_data                                = [word, 'P' * len(word)]
    input_batch, output_batch, target_batch = make_batch([seq_data])
    # 결과가 [batch size, time step, input] 으로 나오기 때문에,
    # 2번째 차원인 input 차원을 argmax 로 취해 가장 확률이 높은 글자를 예측 값으로 만든다.
    prediction = tf.argmax(model, 2)
    result     = sess.run(prediction,
                          feed_dict={enc_input: input_batch,
                                     dec_input: output_batch,
                                     targets: target_batch})
    decoded    = [char_arr[i]    for i in result[0]] # 숫자의 인덱스 해당하는 글자를 가져와 글자 배열을 만든다
    end        = decoded.index('E')                  # 출력 끝의 'E' 이후 글자를 제거 후 문자열로 만든다.
    translated = ''.join(decoded[:end])
    return translated

In [7]:
# print Translate words
words = ['word','game','fight','loev','abcd']
print('\n=== 번역 테스트 ===')
for word in words:
    print(word, '-->', translate(word))


=== 번역 테스트 ===
word --> 단어
game --> 놀이
fight --> 키스
loev --> 사랑
abcd --> 사랑


<br><br>
# **Sequence To Sequence Model for Image**
Simple LSTM

In [5]:
! pip install imageio

    100% |████████████████████████████████| 3.3MB 5.9MB/s eta 0:00:01


In [12]:
import numpy as np
import tensorflow as tf
from scipy.misc import imresize
from imageio import imread, imsave
img      = imread('./data/코끼리_tinted.jpg')
img1     = imread('./data/햄스터_tinted.jpg')
data     = [img, img1]
char_arr = [c for c in 'SEP가아나기다코구끼라리와마오리바햄스사터뒤에물통 ']
num_dic  = {n: i for i, n in enumerate(char_arr)}
dic_len  = len(num_dic)+1
max      = 15 #입력과 출력의 최대 글자 수

In [13]:
# Convolution 2d layer
def conv2d(inputs, filters, kernel_size=(3, 3), strides=(1, 1),
           activation=tf.nn.relu, use_bias=True, name=None):    
    return tf.layers.conv2d(inputs   = inputs,   filters     = filters,
                            strides  = strides,  kernel_size = kernel_size, 
                            padding  = 'same',   activation  = activation,
                            use_bias = use_bias, name        = name)

# 2D Max Pooling layer.
def max_pool2d(inputs, pool_size = (2, 2), strides = (2, 2), name = None):
    return tf.layers.max_pooling2d(inputs  = inputs,  pool_size = pool_size,
                                   strides = strides, padding   = 'same',
                                   name    = name)

# Fully-connected layer
def dense(inputs, units, activation = tf.tanh, use_bias = True, name = None):
    return tf.layers.dense(inputs = inputs,  units = units, name = name, 
                           activation = activation, use_bias = use_bias)

In [14]:
###사진의 크기 224*224
data   = np.array(data)
images = tf.placeholder( dtype=tf.float32, shape=[None,224,224,3])

In [16]:
# Build the VGG16 net
def build_vgg16(images):
    conv1_1_feats = conv2d(images, 64, name='conv1_1')
    conv1_2_feats = conv2d(conv1_1_feats, 64, name='conv1_2')
    pool1_feats   = max_pool2d(conv1_2_feats, name='pool1')    
    conv2_1_feats =conv2d(pool1_feats, 128, name='conv2_1')
    conv2_2_feats = conv2d(conv2_1_feats, 128, name='conv2_2')
    pool2_feats   = max_pool2d(conv2_2_feats, name='pool2')    
    conv3_1_feats = conv2d(pool2_feats, 256, name='conv3_1')
    conv3_2_feats = conv2d(conv3_1_feats, 256, name='conv3_2')
    conv3_3_feats = conv2d(conv3_2_feats, 256, name='conv3_3')
    pool3_feats   = max_pool2d(conv3_3_feats, name='pool3')    
    conv4_1_feats = conv2d(pool3_feats, 256, name='conv4_1')
    conv4_2_feats = conv2d(conv4_1_feats, 256, name='conv4_2')
    conv4_3_feats = conv2d(conv4_2_feats, 256, name='conv4_3')
    pool4_feats   = max_pool2d(conv4_3_feats, name='pool4') 
    conv5_1_feats = conv2d(pool4_feats, 256, name='conv5_1')
    conv5_2_feats = conv2d(conv5_1_feats, 256, name='conv5_2')
    conv5_3_feats = conv2d(conv5_2_feats, 256, name='conv5_3')#14*14*256 으로 출력
    reshaped_conv5_3_feats = tf.reshape(conv5_3_feats, [-1, 196*256])
    den = dense(reshaped_conv5_3_feats,4096)
    den = dense(den, 4096)
    den = dense(den, 4096)
    denima = build_vgg16(images)
    return denima 

In [19]:
def make_batch(seq_data):
    output_batch = []
    target_batch = []
    for seq in seq_data:        # 디코더 셀의 입력값. 시작을 나타내는 S 심볼을 맨 앞에 붙여준다.        
        output = [num_dic[n] for n in ('S' + seq[1])]        # 학습을 위해 비교할 디코더 셀의 출력값. 끝나는 것을 알려주기 위해 마지막에 E 를 붙인다.
        target = [num_dic[n] for n in (seq[1] + 'E')]        ##아래는 글자의 최대수보다 낮으면 패딩으로 채워주고 크면 최대만큼 잘라냄
        if len(output) <= max:
            for i in range(max - len(output)):
                output.append(2)
        else:
            output = output[:max]        
    if len(target) <= max:
        for i in range(max - len(target)):
            target.append(2)
    else:
        target = target[:max]
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)    
    return  output_batch, target_batch

In [21]:
# 입력 데이터는 필요 없기 때문에 비워둔다
learning_rate = 0.0005
label         = [['','코끼리와 오리'],['','햄스터와 물통']]

# CNN출력의 사이즈와 맞춘다
n_hidden    = 4096 
total_epoch = 200
n_class     = n_input = dic_len
dec_input   = tf.placeholder(tf.float32, [None, None, n_input])

In [22]:
# [batch size, time steps]
targets = tf.placeholder(tf.int64, [None, None])
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    # Seq2Seq 모델에서는 인코더 셀의 최종 상태값을 디코더 셀의 초기 상태값으로 넣어주지만 이 모델에서는 CNN에서 나온 출력값을 초기상태값으로 입력한다
    dec_cell  = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)        
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state=ima, dtype=tf.float32) # 초기상태값
    model     = tf.layers.dense(outputs, n_class, activation=None) # 단어 갯수만큼 출력을 맞춰줌
    cost      = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets))
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

ValueError: Variable decode/rnn/basic_rnn_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-2-d8a6e6087995>", line 11, in <module>
    dtype=tf.float32)
  File "/home/erdos/Jupyter/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/erdos/Jupyter/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2903, in run_ast_nodes
    if self.run_code(code, result):


In [23]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
output_batch, target_batch = make_batch(label)

def translate(imge):    
    seq_data = ['',''] # 처음 데이터로 시작을 뜻하는 S만들어가면 되기때문에 둘다 비워둠    
    output_batch, target_batch = make_batch([seq_data]) # S만 들어가있는 데이터
    imge = [imge]    # 결과가 [batch size, time step, input] 으로 나오기 때문에,
    # 2번째 차원인 input 차원을 argmax 로 취해 가장 확률이 높은 글자를 예측 값으로 만든다.
    prediction = tf.argmax(model, 2)    
    result = sess.run(prediction,feed_dict={images :imge, dec_input: output_batch})
    # 여기서 S 다음에 올 글자 예측 
    # 원래는 디코더 입력 데이터를 다 패딩처리해서 한꺼번에 넣어주었는데
    # 여기서는 다음단어를 예측하여 그다음 LSTM에 넣어줌    
    for i in range(max):
        if result[0][i]==1: # 끝을 뜻하는 E가 번호로 1로 표현되기 때문에 1이나오면 종료            
            break
        elif i == 0: # result에 처음 데이터는 S다음 데이터이기 때문에 다음 output으로 S를 뜻하는 0과 re 를 같이 넣어줌
            re = result[0][i]
            output = [0,re]
            output_batch = []
            output_batch.append(np.eye(dic_len)[output])
            output = output[:2]
            result = sess.run(prediction, 
                              feed_dict={images    : imge,
                                         dec_input : output_batch})        
        # 계속해서 result의 출력을 아웃풋에 추가시켜 단어를 예측
        else: 
            re = result[0][i]
            output.append(re)
            output_batch = []
            output_batch.append(np.eye(dic_len)[output])
            output = output[:i+2]
            result = sess.run(prediction,
                              feed_dict={images    : imge,
                                         dec_input : output_batch})
    if i == max-1:##최대길이면 그냥 출력
        break    # 결과 값인 숫자의 인덱스에 해당하는 글자를 가져와 글자 배열을 만든다.
    # 출력의 끝을 의미하는 'E' 이후의 글자들을 제거하고 문자열로 만든다.
    decoded = [char_arr[i] for i in result[0]]    
    if 'E' in decoded:
        end        = decoded.index('E')
        translated = ''.join(decoded[:end])
    # E가 없으면 모두출력
    else: 
        translated = ''.join(decoded)    
    return translated

SyntaxError: 'break' outside loop (<ipython-input-23-18413f59caaf>, line 38)

In [ ]:
for epoch in range(total_epoch):    
    _, loss = sess.run([optimizer, cost], 
                       feed_dict={images    : data,
                                  dec_input : output_batch,
                                  targets   : target_batch})    
    print('Epoch:', '%04d' % (epoch + 1),
         'cost =', '{:.6f}'.format(loss))
    if epoch % 10 == 0:  # 훈련결과를 10번마다 확인
        print('img ->', translate(img))